<a href="https://colab.research.google.com/github/annasu1225/AlphaFold2_missesense_mutations/blob/main/Deleting_Missing_Residues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook finds the missing residues based on REMARK 465 in the reference PDB and deletes these residues in the model PDB.

Anna
Dec 5, 2022

Import BioPython's PDB package.

In [ ]:
!pip install biopython
import Bio.PDB as bpdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 8.6 MB/s 


Connect to your Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Get mutant information

In [ ]:
mutant_ID = '1jbq'
chain_id = 'A'

Get the reference PDB and model PDB



In [ ]:
import os.path
root = '/content/drive/My Drive/Colab Notebooks/'
ref_pdb = os.path.join(root, 'reference', f'{mutant_ID}_reference.pdb')
model_pdb = os.path.join(root, 'unrelaxed_predictions', mutant_ID, f'{mutant_ID}_model_uncropped.pdb')

Get the structure

In [ ]:
s = bpdb.PDBParser().get_structure('temp', model_pdb)

Get the positions of the missing residues

In [ ]:
fin = open(ref_pdb,'r')
ref = fin.readlines()
fin.close()

mis_res = []

for i,line in enumerate(ref):
  if line[0:10] == 'REMARK 465':
    split = [line[:10], line[19], line[21:26]]
    if split[1] == chain_id:
      mis_res.append(int(split[2]))
  if line[0:5] == 'DBREF':
    seqBegin = int(line[14:18])

missed_res_indices = sorted(list(set(mis_res)))

In [ ]:
for i in range(len(missed_res_indices)):
    missed_res_indices[i] = missed_res_indices[i] - seqBegin + 1
print(seqBegin)
print(missed_res_indices)

-21
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 216, 217, 218, 219, 220, 221, 222, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435]


Define a function to exclude the missing residues in the model PDB

In [ ]:
class ResSelect(bpdb.Select):
    def accept_residue(self, res):
        if res.id[1] in missed_res_indices and res.parent.id == chain_id:
            return False
        else:
            return True

Save the new model PDB file 

In [ ]:
io = bpdb.PDBIO()
io.set_structure(s)
io.save(f"{mutant_ID}_model.pdb", ResSelect())

References: 
1. https://stackoverflow.com/questions/44151021/gap-characters-corresponding-to-the-missing-residues-in-a-protein-sequence-using
2.https://stackoverflow.com/questions/22452748/remove-parts-from-a-pdb-file-using-python